In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from datetime import datetime
import numpy as np

In [2]:
spark = SparkSession.builder.appName('hw_2022').master('local[*]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/13 06:13:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/13 06:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/01/13 06:13:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Question 1. Install Spark and PySpark
# Execute spark.version
# What's the output?

spark.version

#### Answer: 3.3.2

'3.3.2'

In [4]:
# Question 2. HVFHW February 2021
# Repartition it to 24 partitions and save it to parquet.
# What's the size of the folder with results (in MB)?

!curl -Lf https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet -o hw_2021.parquet
df = spark.read.option('header', 'true').option('inferSchema', 'true').parquet('hw_2021.parquet')
df.repartition(24).write.parquet('hw_2022')
!du -hs hw_2022/

#### Answer: 516M

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  288M  100  288M    0     0  16.4M      0  0:00:17  0:00:17 --:--:-- 18.5M


[Stage 3:======================================================>  (23 + 1) / 24]

516M	hw_2022/


In [5]:
# Question 3. Count records
# How many taxi trips were there on February 15?
# Consider only trips that started on February 15.


df = df.filter((df.pickup_datetime >= datetime(2021, 2, 1)) & (df.pickup_datetime < datetime(2021, 3, 1)))
df.filter(F.to_date(df.pickup_datetime) == datetime(2021, 2, 15)).count()

#### Answer: 367170

367170

In [6]:
# Question 4. Longest trip for each day
# Now calculate the duration for each trip.
# Trip starting on which day was the longest?

df.groupby(F.to_date(df.pickup_datetime).alias('date')).\
    agg(F.max('trip_miles').alias('max_miles')).\
    sort('max_miles', ascending = False).\
    limit(5).show()

#### Answer: 527.11

[Stage 7:============================================>              (3 + 1) / 4]

+----------+---------+
|      date|max_miles|
+----------+---------+
|2021-02-18|   527.11|
|2021-02-10|    512.5|
|2021-02-09|   480.73|
|2021-02-27|   454.49|
|2021-02-22|   347.41|
+----------+---------+



In [7]:
# Question 5. Most frequent dispatching_base_num
# Now find the most frequently occurring dispatching_base_num in this dataset.
# How many stages this spark job has?

df.groupby('dispatching_base_num').\
    count().sort('count', ascending = False).\
    limit(5).show()

### Answer: 2, one for grouping and one for sorting.

[Stage 10:===========================================>              (3 + 1) / 4]

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02510|3233664|
|              B02764| 965568|
|              B02872| 882689|
|              B02875| 685390|
|              B02765| 559768|
+--------------------+-------+



In [8]:
# Question 6. Most common locations pair
# Find the most common pickup-dropoff pair.
# For example:
# "Jamaica Bay / Clinton East"
# Enter two zone names separated by a slash
# If any of the zone names are unknown (missing), use "Unknown". For example, "Unknown / Clinton East".

!curl -Lf https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv -o z.csv
zone = spark.read.option('header', 'true').option('inferSchema', 'true').csv('z.csv')
zone = spark.read.option('header', 'true').option('inferSchema', 'true').csv('z.csv')
joined = df.join(zone, df.PULocationID == zone.LocationID, how = 'left')
joined = joined.drop('LocationID')
joined = joined.\
    withColumnRenamed('Borough', 'pickup_borough').\
    withColumnRenamed('Zone', 'pickup_zone').\
    withColumnRenamed('service_zone', 'pickup_service_zone')
joined = joined.join(zone, df.DOLocationID == zone.LocationID, how = 'left')
joined = joined.\
    withColumnRenamed('Borough', 'dropoff_borough').\
    withColumnRenamed('Zone', 'dropoff_zone').\
    withColumnRenamed('service_zone', 'dropoff_service_zone')
joined = joined.drop('LocationID')
joined.groupBy(
    F.coalesce(F.col("pickup_zone"), F.lit("Unknown")).alias("pickup_zone"),
    F.coalesce(F.col("dropoff_zone"), F.lit("Unknown")).alias("dropoff_zone")
).count() \
 .orderBy(F.desc("count")).limit(5).show()

#### Answer: East New York / East New York

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 12322  100 12322    0     0  24219      0 --:--:-- --:--:-- --:--:-- 24219


[Stage 18:===========================================>              (3 + 1) / 4]

+-------------------+-------------------+-----+
|        pickup_zone|       dropoff_zone|count|
+-------------------+-------------------+-----+
|      East New York|      East New York|45041|
|       Borough Park|       Borough Park|37329|
|           Canarsie|           Canarsie|28026|
|Crown Heights North|Crown Heights North|25976|
|          Bay Ridge|          Bay Ridge|17934|
+-------------------+-------------------+-----+



In [9]:
spark.stop()
!rm -rf *.csv *.parquet hw_*